In [25]:
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
#!aws s3 cp s3://sagemaker-mumbai-data-set data_set --recursive

In [27]:
!ls data_set/ena_data/

train  validation


In [28]:
IMG_SIZE = 160
BATCH_SIZE = 40

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

INPUT_TENSOR_NAME = "inputs_input"

In [29]:
    base_model = InceptionV3(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=5, activation='softmax'))

In [30]:
tf.keras.models.save_model(
    model,
    '/opt/',
    overwrite=True,
    save_format='tf'
)

NotImplementedError: Saving the model as SavedModel is not supported in TensorFlow 1.Xgraph mode. Please enable eager execution or use the "h5" save format.

In [17]:
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [42]:
model.input

<tf.Tensor 'inception_v3_input_2:0' shape=(?, 160, 160, 3) dtype=float32>

In [19]:
model.model

AttributeError: 'Sequential' object has no attribute 'model'

In [4]:
def keras_model_fn(hyperparameters):
    base_model = InceptionV3(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=256, activation='relu'))
    model.add(layers.Dense(units=5, activation='softmax'))
    
    adamOptimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer=adamOptimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
    return model

In [5]:
model = keras_model_fn(hyperparameters=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
len(model.trainable_weights)

4

In [7]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
generator = datagen.flow_from_directory('data_set/ena_data/train', target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)

Found 4000 images belonging to 5 classes.


In [9]:
validation_generator = datagen.flow_from_directory(
        'data_set/ena_data/validation', target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)

Found 500 images belonging to 5 classes.


In [10]:
model.fit_generator(
        generator,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=800)

 99/100 [============================>.] - ETA: 1s - loss: 1.7710 - acc: 0.5487

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 1260s 13s/step - loss: 1.7636 - acc: 0.5493 - val_loss: 1.2828 - val_acc: 0.6979
